In [2]:
import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.6.0'

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

In [5]:
train_datagen = ImageDataGenerator(rotation_range=90,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                   )

test_datagen = ImageDataGenerator(validation_split=0.2)

valid_datagen = ImageDataGenerator(validation_split=0.5)

In [ ]:
disease_name = input('원하는 질병 이름을 입력하세요: ')

In [6]:
data_dir = '1.Training/라벨링데이터/TL1/개/안구/일반/' + disease_name

In [7]:
train_generator = train_datagen.flow_from_directory(data_dir,
                                                    target_size = (224,224),
                                                    batch_size=32,
                                                    class_mode='sparse',
                                                    seed=42,
                                                    subset='training')

test_generator = test_datagen.flow_from_directory(data_dir,
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='sparse',
                                                  seed=42,
                                                  subset='validation')

Found 20621 images belonging to 3 classes.
Found 4124 images belonging to 3 classes.


In [9]:
labels = (train_generator.class_indices)
n_classes = len(labels)

{'무': 0, '상': 1, '하': 2}

In [10]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a GlobalAveragePooling2D layer to reduce the spatial dimensions
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with a single output for binary classification
x = Dense(n_classes, activation='softmax')(x)

# Create the final model by attaching the binary classification head to the base model
model = Model(inputs=base_model.input, outputs=x)

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

saved_model = disease_name + '_DenseNet.h5'

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5,restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint(saved_model, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [17]:
history = model.fit(train_generator,
          epochs=50,
          validation_data=test_generator,
        #   steps_per_epoch=len(train_generator),
        #   validation_steps=len(test_generator),
          callbacks=[es, mc]
          )

Epoch 1/50
645/645 [==============================] - 251s 369ms/step - loss: 0.3760 - accuracy: 0.8429 - val_loss: 5.4334 - val_accuracy: 0.5754

Epoch 00001: val_accuracy improved from -inf to 0.57541, saving model to 궤양성각막질환_DenseNet.h5
Epoch 2/50
645/645 [==============================] - 170s 264ms/step - loss: 0.2766 - accuracy: 0.8867 - val_loss: 0.3931 - val_accuracy: 0.8358

Epoch 00002: val_accuracy improved from 0.57541 to 0.83584, saving model to 궤양성각막질환_DenseNet.h5
Epoch 3/50
645/645 [==============================] - 165s 256ms/step - loss: 0.2443 - accuracy: 0.9022 - val_loss: 0.3281 - val_accuracy: 0.8797

Epoch 00003: val_accuracy improved from 0.83584 to 0.87973, saving model to 궤양성각막질환_DenseNet.h5
Epoch 4/50
157/645 [======>.......................] - ETA: 1:58 - loss: 0.2168 - accuracy: 0.9141

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(epochs, history.history['loss'], 'b', label='train_loss')
plt.plot(epochs, history.history['val_loss'], 'g', label='val_loss')
plt.ylim([0,1.0])
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.subplot(1, 2, 2)
plt.title('Accuray')
plt.plot(epochs, history.history['accuracy'], 'b', label='train_accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'g', label='val_accuracy')
plt.ylim([0.7,1])
plt.grid(True)
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()